In [4]:
import requests
import pprint as pp
import datetime

from urllib.parse import urljoin 
from getpass import getpass
from requests.auth import HTTPBasicAuth

The Ovation web API is a RESTful API that you use with any HTTP client. In this tutorial, we'll use the Python `requests` package.

### Utilities

In [23]:
API_HOST = 'http://127.0.0.1:3000' #'https://api-dev.ovation.io/'
def make_url(path, host=API_HOST):
    return urljoin(host, path)

In [14]:
make_url('/api/v1/projects')

'http://0.0.0.0:3000/api/v1/projects'

### Credentials

Enter your Ovation API key:

In [16]:
api_key = getpass(prompt="API Key: ")

API Key: ········


We create a `requests.Session` object for use with the Ovation API

In [34]:
def make_session(api_key=api_key):
    auth_params = {'api-key' : api_key}
    
    api = requests.Session()
    api.verify = True # Verify SSL certificates
    api.headers['content-type'] = 'application/json'
    api.headers['authorization'] = 'Token token={}'.format(api_key)
    
    return api

api = make_session(api_key)

### Projects

In [77]:
make_url('/api/v1/projects')

'http://127.0.0.1:3000/api/v1/projects'

In [78]:
r = api.get(make_url('/api/v1/projects'))
r.raise_for_status()

In [79]:
projects = r.json()['projects']

In [80]:
len(projects)

10

In [81]:
pp.pprint(projects[0])

{'_id': '52b6366a-1528-4bf9-a091-09c4e935d1cd',
 '_rev': '1-90c4459b197e1917c171d0cf38ee694c',
 'attributes': {'name': 'abby'},
 'links': {'_collaboration_roots': ['52b6366a-1528-4bf9-a091-09c4e935d1cd'],
           'analysis_records': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/analysis_records',
           'experiments': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/experiments',
           'notes': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/annotations/notes',
           'owner': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/links/owner',
           'properties': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/annotations/properties',
           'self': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd',
           'tags': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/annotations/tags',
           'timeline-events': '/api/v1/entities/52b6366a-1528-4bf9-a091-09c4e935d1cd/annotations/timeline-events'},
 'named_l

## Creating links

In [ ]:
r = api.post

### Following links

In [84]:
r = api.get(make_url(projects[0]['links']['experiments']))
r.raise_for_status()

In [85]:
experiments = r.json()['rel']
pp.pprint(experiments)

[]


In [72]:
exp = experiments[0]
pp.pprint(exp)

IndexError: list index out of range

In [48]:
for p in projects:
    experiments = api.get(make_url(p['links']['experiments'])).json()
    for exp in experiments:
        pp.pprint(api.get(make_url(exp['links']['epoch_groups'])).json())
        pp.pprint(api.get(make_url(exp['links']['epochs'])).json())

TypeError: string indices must be integers

### Adding analysis record

In [92]:
# TODO we should define the types somewhere
est = pytz.timezone('US/Eastern')
d = datetime.datetime.now(est)

new_record = {'type': 'AnalysisRecord',
              'attributes': {'name': 'Tutorial Analysis',
                              'date': d.isoformat()},
              'links': {'parent': [{'target_id': projects[0]['_id'],
                                    'inverse_rel': 'analysis_records'}]}
              }

r = api.post(make_url('/api/v1/entities'), data=json.dumps(new_record))
r.json()

[{'_id': '0fd8fd6e-d996-4595-a547-745e949c98af',
  'links': {'self': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af',
   'parent': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af/links/parent',
   'owner': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af/links/owner',
   '_collaboration_roots': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af/links/_collaboration_roots'},
  'attributes': {'name': 'Tutorial Analysis',
   'date': '2014-10-10T13:32:26.077167-04:00'},
  'type': 'AnalysisRecord'}]

In [93]:
ar = r.json()[0]

In [66]:
r.status_code

201

Refresh projects so that we get the up-to-date links

In [77]:
projects[0]['links']['self']

'/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6'

In [84]:
p = api.get(make_url(projects[0]['links']['self'])).json()['entity']

In [85]:
p

{'_id': '7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6',
 'named_links': {},
 'attributes': {'name': 'My First Project',
  'purpose': 'First Project',
  'start': '2014-09-25T22:18:19.724-04:00'},
 'type': 'Project',
 'links': {'experiments': '/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6/links/experiments',
  'self': '/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6',
  'owner': '/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6/links/owner',
  '_collaboration_roots': '/api/v1/entities/7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6/links/_collaboration_roots'},
 '_rev': '3-b0ede23a375c31c56663960447319237'}

In [94]:
api.get(make_url(ar['links']['self'])).json()

[{'_id': '0fd8fd6e-d996-4595-a547-745e949c98af',
  'links': {'self': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af',
   'parent': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af/links/parent',
   'owner': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af/links/owner',
   '_collaboration_roots': '/api/v1/entities/0fd8fd6e-d996-4595-a547-745e949c98af/links/_collaboration_roots'},
  'attributes': {'name': 'Tutorial Analysis',
   'date': '2014-10-10T13:32:26.077167-04:00'},
  'type': 'AnalysisRecord',
  '_rev': '1-bd7b00dcd33ed2cb8a387745b6133896'}]

In [98]:
# TODO p['links']['analysis_records']
api.get(make_url('/api/v1/entities/{}/links/analysis_records'.format(p['_id']))).json()

[]

'7cebfee3-ee3f-4f34-a0dc-e4e7f79a18a6'